In [1]:
import collections
import re
from d2l import torch as d2l

In [64]:
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():
    """将时间机器数据集加载到文本行的列表中"""
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine()
print(f'# 文本总行数: {len(lines)}')
print(lines[0])
print(lines[10])

# 文本总行数: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


In [65]:
def tokenize(lines, token='word'):
    """"将文本行拆分为单词或字符标记"""
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('错误：未知词元类型：' + token)

In [66]:
tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])


['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


In [67]:
def count_corpus(tokens):
    """统计词元的频率"""
    # 这里的tokens是1D列表或2D列表
    if len(tokens) == 0 or isinstance(tokens[0], list):
        # 将词元列表展平成一个列表
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

In [71]:
# vocabulary
# 用来将字符串类型的标记映射到从0开始的数字索引中
class Vocab:
    """文本词汇表"""
    # min_freq: 如果一个token出现次数小于min_freq，则丢掉
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens == None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        # 按出现频率排序
        counter = count_corpus(tokens)
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1],reverse=True)
        # 未知词元的索引为0
        self.idx_to_token = ['<unk>'] + reserved_tokens
        self.token_to_index = {token: idx for idx, token in enumerate(self.idx_to_token)}
        for token, freq in self._token_freqs:
            if freq < min_freq:
                # break是因为排过序了..
                break
            if token not in self.token_to_index:
                # 按照词频一个一个往里面加，idx就是其所在的位置
                self.idx_to_token.append(token)
                self.token_to_index[token] = len(self.idx_to_token)-1
    def __len__(self):
        return len(self.idx_to_token)
    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_index.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]
    def to_token(self, indices):
        if not isinstance(tokens, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]
    @property
    def unk(self):  # 未知词元的索引为0
        return 0
    @property
    def token_freqs(self):
        return self._token_freqs


In [77]:
vocab = Vocab(tokens=tokens)
list(vocab.token_to_index.items())[:10]

[('<unk>', 0),
 ('the', 1),
 ('i', 2),
 ('and', 3),
 ('of', 4),
 ('a', 5),
 ('to', 6),
 ('was', 7),
 ('in', 8),
 ('that', 9)]

In [79]:
for i in [0, 10]:
    print(i)
    print('words:', tokens[i])
    print('indices:', vocab[tokens[i]])

0
words: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
indices: [1, 19, 50, 40, 2183, 2184, 400]
10
words: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
indices: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


In [38]:
a = [['a', 'b', 'a'],['b', 'b', 'c']]
b = [t for lines in a for t in lines]
print(b)
m = collections.Counter(b)
m, m.items()

['a', 'b', 'a', 'b', 'b', 'c']


(Counter({'b': 3, 'a': 2, 'c': 1}), dict_items([('a', 2), ('b', 3), ('c', 1)]))

In [42]:
# lambda表达式里的x[i]表示根据第i+1个元素排序。因为是从0开始的
t = sorted(m.items(), key=lambda x:x[1], reverse=True)
t

[('b', 3), ('a', 2), ('c', 1)]

In [49]:
for i, j in t:
    print(i)
    print(j)
    print('###')

b
3
###
a
2
###
c
1
###


In [47]:
lst = [('candy',5,100), ('apple',10,200), ('baby',20,300)]
lst.sort(key=lambda x:x[2])
print(lst)

[('candy', 5, 100), ('apple', 10, 200), ('baby', 20, 300)]


In [55]:
reserved_tokens = ['a', 'b']
idx_to_token = ['<unk>'] + reserved_tokens
token_to_index = {token: idx for idx, token in enumerate(idx_to_token)}
token_to_index, 'a' in token_to_index, 1 in token_to_index, token_to_index.get('c', 0), token_to_index.get('a', 0)

({'<unk>': 0, 'a': 1, 'b': 2}, True, False, 0, 1)

In [80]:
# 整合所有功能

def load_corpus_time_machine(max_tokens=-1):  #@save
    """返回时光机器数据集的词元索引列表和词表"""
    lines = read_time_machine()
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    # 因为时光机器数据集中的每个文本行不一定是一个句子或一个段落，
    # 所以将所有文本行展平到一个列表中
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)

(170580, 28)